In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
titanic = pd.read_csv("train_titanic.csv")

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
features = titanic.columns.tolist()[2:]

In [5]:
features

['Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
features.pop(features.index('Name'))

'Name'

In [7]:
features.pop(features.index('Ticket'))
features.pop(features.index('Cabin'))

'Cabin'

In [8]:
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [9]:
from sklearn.model_selection import train_test_split
X = pd.concat([titanic[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']], pd.get_dummies(titanic[['Sex', 'Embarked']])], axis=1)
y = titanic.Survived
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
features = X.columns.tolist()

In [11]:
for i in features:
    print i + '    :    ' + str(sum(X_train[i].isnull()))

Pclass    :    0
Age    :    119
SibSp    :    0
Parch    :    0
Fare    :    0
Sex_female    :    0
Sex_male    :    0
Embarked_C    :    0
Embarked_Q    :    0
Embarked_S    :    0


In [12]:
mediana = X_train.Age.median()

X_train.loc[X_train.Age.isnull(),'Age'] = mediana
X_test.loc[X_test.Age.isnull(),'Age'] = mediana

C:\Users\Big\Anaconda2\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
for i in features:
    print i + '    :    ' + str(sum(X_train[i].isnull()))

Pclass    :    0
Age    :    0
SibSp    :    0
Parch    :    0
Fare    :    0
Sex_female    :    0
Sex_male    :    0
Embarked_C    :    0
Embarked_Q    :    0
Embarked_S    :    0


In [14]:
for i in features:
    print i
    print X_train.groupby(y_train)[i].describe()

Pclass
Survived       
0         count    397.000000
          mean       2.526448
          std        0.740209
          min        1.000000
          25%        2.000000
          50%        3.000000
          75%        3.000000
          max        3.000000
1         count    226.000000
          mean       1.977876
          std        0.856062
          min        1.000000
          25%        1.000000
          50%        2.000000
          75%        3.000000
          max        3.000000
Name: Pclass, dtype: float64
Age
Survived       
0         count    397.000000
          mean      29.743073
          std       12.487801
          min        1.000000
          25%       23.000000
          50%       28.000000
          75%       34.500000
          max       74.000000
1         count    226.000000
          mean      28.233407
          std       13.838029
          min        0.670000
          25%       22.000000
          50%       28.000000
          75%       35.00000

In [15]:
print X_train.shape
print X_test.shape

(623, 10)
(268, 10)


In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [17]:
# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

In [19]:
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [20]:
predictions = mlp.predict(X_test)

In [21]:
probs = mlp.predict_proba(X_test)

In [22]:
from sklearn.metrics import roc_auc_score
auc_ = roc_auc_score(y_test, probs[:,1])
print("AUC: %.4f" % auc_)

AUC: 0.8764


In [23]:
from sklearn.metrics import accuracy_score
print("acurácia: %.4f" % accuracy_score(y_test, predictions))

acurácia: 0.8097


In [24]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
print classification_report(y_test,predictions)

[[136  16]
 [ 35  81]]
             precision    recall  f1-score   support

          0       0.80      0.89      0.84       152
          1       0.84      0.70      0.76       116

avg / total       0.81      0.81      0.81       268



In [25]:
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [26]:
tuned_parameters = [{'hidden_layer_sizes': [(1,), (5,), (10,), (5,5,)],
                     'activation' : ['identity', 'logistic', 'tanh', 'relu'],
                     'learning_rate': ['constant', 'adaptive'],
                     'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}]

In [27]:
clf = GridSearchCV(MLPClassifier(), tuned_parameters, cv=3)
clf.fit(X_train, y_train)

C:\Users\Big\Anaconda2\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


GridSearchCV(cv=3, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'alpha': [0.0001, 0.001, 0.01, 0.1, 1], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'learning_rate': ['constant', 'adaptive'], 'hidden_layer_sizes': [(1,), (5,), (10,), (5, 5)]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [28]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, clf.predict(X_test)))
print()

Best parameters set found on development set:
()
{'alpha': 0.1, 'activation': 'relu', 'learning_rate': 'constant', 'hidden_layer_sizes': (5, 5)}
()
Grid scores on development set:
()
0.657 (+/-0.416) for {'alpha': 0.0001, 'activation': 'identity', 'learning_rate': 'constant', 'hidden_layer_sizes': (1,)}
0.767 (+/-0.052) for {'alpha': 0.0001, 'activation': 'identity', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (1,)}
0.795 (+/-0.034) for {'alpha': 0.0001, 'activation': 'identity', 'learning_rate': 'constant', 'hidden_layer_sizes': (5,)}
0.790 (+/-0.042) for {'alpha': 0.0001, 'activation': 'identity', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (5,)}
0.801 (+/-0.059) for {'alpha': 0.0001, 'activation': 'identity', 'learning_rate': 'constant', 'hidden_layer_sizes': (10,)}
0.798 (+/-0.038) for {'alpha': 0.0001, 'activation': 'identity', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (10,)}
0.796 (+/-0.042) for {'alpha': 0.0001, 'activation': 'identity', 'learning_rate': '

In [29]:
params = {'n_estimators': 500, 'max_depth': 2, 'min_samples_split': 2,
          'learning_rate': 0.01}
clf = ensemble.GradientBoostingClassifier(**params)

clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

             precision    recall  f1-score   support

          0       0.77      0.91      0.83       152
          1       0.84      0.65      0.73       116

avg / total       0.80      0.79      0.79       268



In [30]:
tuned_parameters = [{'n_estimators': [10, 100, 300],
                     'max_depth' : [3, 10],
                     'min_samples_split': [2, 10],
                     'learning_rate': [0.001, 0.1], 
                     'subsample': [0.5, 1]}]

In [31]:
clf = GridSearchCV(ensemble.GradientBoostingClassifier(), tuned_parameters, cv=5)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [10, 100, 300], 'min_samples_split': [2, 10], 'learning_rate': [0.001, 0.1], 'max_depth': [3, 10], 'subsample': [0.5, 1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [32]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, clf.predict(X_test)))
print()

Best parameters set found on development set:
()
{'min_samples_split': 2, 'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3, 'subsample': 1}
()
Grid scores on development set:
()
0.637 (+/-0.003) for {'min_samples_split': 2, 'n_estimators': 10, 'learning_rate': 0.001, 'max_depth': 3, 'subsample': 0.5}
0.637 (+/-0.003) for {'min_samples_split': 2, 'n_estimators': 10, 'learning_rate': 0.001, 'max_depth': 3, 'subsample': 1}
0.637 (+/-0.003) for {'min_samples_split': 2, 'n_estimators': 100, 'learning_rate': 0.001, 'max_depth': 3, 'subsample': 0.5}
0.637 (+/-0.003) for {'min_samples_split': 2, 'n_estimators': 100, 'learning_rate': 0.001, 'max_depth': 3, 'subsample': 1}
0.790 (+/-0.056) for {'min_samples_split': 2, 'n_estimators': 300, 'learning_rate': 0.001, 'max_depth': 3, 'subsample': 0.5}
0.807 (+/-0.046) for {'min_samples_split': 2, 'n_estimators': 300, 'learning_rate': 0.001, 'max_depth': 3, 'subsample': 1}
0.637 (+/-0.003) for {'min_samples_split': 10, 'n_estimators': 10, 'lea

In [33]:
from sklearn.svm import SVC

In [34]:
clf = SVC(C=1.0)

In [35]:
clf.fit(X_train, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [36]:
print(classification_report(y_test, clf.predict(X_test)))

             precision    recall  f1-score   support

          0       0.78      0.93      0.85       152
          1       0.87      0.66      0.75       116

avg / total       0.82      0.81      0.80       268



In [ ]:
tuned_parameters = [{'kernel': ['rbf', 'linear', 'poly'],
                     'C': [1, 10, 100, 1000]}]

In [ ]:
clf = GridSearchCV(SVC(), tuned_parameters, cv=5)
clf.fit(X_train, y_train)

In [ ]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, clf.predict(X_test)))
print()